In [3]:
import os, glob
import pandas as pd
import pysam
import numpy as np
import pickle

In [4]:
cancer_type = "Brain"
non_coding_region="core_prom"
intersected_data = "/data/projects/GDC_Cancer_Wise/New_data/{}/Generated_files/Intersected_Data/Germline/non_coding_regions/{}/intersected_vcf_{}_data.pkl".format(cancer_type, non_coding_region, non_coding_region)
reference_path  = "/data/projects/Resources/Gencode_genome_annotation/GRCh38.primary_assembly.genome.fa"

In [5]:
output_path = "/data/projects/GDC_Cancer_Wise/New_data/{}/Generated_files/DNABERT_Data/Germline/{}/".format(cancer_type, non_coding_region)
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [6]:
def seq2kmer(seq):
    """
    Convert original sequence to kmers
    
    Arguments:
    seq -- str, original sequence.
    k -- int, kmer of length k specified.
    
    Returns:
    kmers -- str, kmers separated by space
    """
    k=6
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers

In [7]:
def get_sequences(df, reference_fasta):
    data = []
    for idx, row in df.iterrows():
        #print(row)
        chrom = row['chr_name']
        ref_start = row['start']
        ref_end = row['end']
        variant_start = row['START_POS']
        variant_end = row['END_POS']
        ref_nucleotide = row['REF']
        alt = row["ALT"]
        
        # Adjust for 0-based indexing in python
        variant_pos_start = variant_start - ref_start
        variant_pos_end = variant_end - ref_start
        #print(ref_nucleotide , alt, variant_pos_start, variant_pos_end)
        
        
        # Get reference sequence
        #print(chrom)
        ref_seq = reference_fasta.fetch(chrom, ref_start, ref_end)
        #print(ref_seq)
        
        # Handle insertion and deletion to get the correct alt sequence
        # Identify if the variant is an insertion or deletion
        if len(ref_nucleotide) < len(alt):  # Insertion
            delete_size =  len(alt) - len(ref_nucleotide)
            #print(variant_pos_start, variant_pos_end ,delete_size)
            alt_seq = ref_seq[:variant_pos_start] + alt + ref_seq[variant_pos_end:len(ref_seq) - delete_size]

        elif len(ref_nucleotide) > len(alt):  # Deletion
            insert_size = len(ref_nucleotide) - len(alt)
            #print(insert_size)
            extra_bases = reference_fasta.fetch(chrom, ref_end, ref_end + insert_size)
            #print(extra_bases)
            alt_seq = ref_seq[:variant_pos_start] + alt + ref_seq[variant_pos_end:] + extra_bases

        else:  # SNV
            alt_seq = ref_seq[:variant_pos_start] + alt + ref_seq[variant_pos_end:]
        # print(ref_seq)
        # print(alt_seq)
        # input()


        data.append({
            'chr': chrom,
            'strand': row['strand'],
            'Transcript_ID': row['transcript_id'],
            'Acceptor_start': row['start'],
            'Acceptor_end': row['end'],
            'varinat_start': variant_start,
            'variant_end': variant_end,
            'ref_neucleotide': ref_nucleotide,
            'alternative_neucleotide': alt,
            'reference_seq': ref_seq,
            'alt_seq': alt_seq
        })
            
    
    # Convert the list of dictionaries to a DataFrame
    new_df = pd.DataFrame(data)
    # print(new_df.shape)
    # print(new_df)
    new_df = new_df.drop_duplicates().reset_index()
    print(new_df.shape)
    print("&*")
    data = []
    merged_list = list(zip(new_df['reference_seq'], new_df['alt_seq']))
    merged_list = [item.upper() for tup in merged_list for item in tup]
    #print(merged_list)
    kmer_lst = list(map(seq2kmer, merged_list))
    df_kmer = pd.DataFrame(kmer_lst, columns=['Sequence'])
    df_kmer['Label'] = np.random.choice([0, 1], size=len(df_kmer))
    print(df_kmer.shape)
    return new_df, df_kmer

In [8]:
## Load the VCF files from pickle
with open(intersected_data, "rb") as file:
    loaded_dictionary = pickle.load(file)
print("****All the VCF files are loaded***")   

****All the VCF files are loaded***


In [9]:
## Load the reference file
reference_fasta = pysam.FastaFile(reference_path)
print("#####The reference file is loaded######")

#####The reference file is loaded######


In [10]:
# List all sequence names
sequence_names = reference_fasta.references
print("Sequence names in the FASTA file:", sequence_names)

Sequence names in the FASTA file: ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22', 'chrX', 'chrY', 'chrM', 'GL000008.2', 'GL000009.2', 'GL000194.1', 'GL000195.1', 'GL000205.2', 'GL000208.1', 'GL000213.1', 'GL000214.1', 'GL000216.2', 'GL000218.1', 'GL000219.1', 'GL000220.1', 'GL000221.1', 'GL000224.1', 'GL000225.1', 'GL000226.1', 'KI270302.1', 'KI270303.1', 'KI270304.1', 'KI270305.1', 'KI270310.1', 'KI270311.1', 'KI270312.1', 'KI270315.1', 'KI270316.1', 'KI270317.1', 'KI270320.1', 'KI270322.1', 'KI270329.1', 'KI270330.1', 'KI270333.1', 'KI270334.1', 'KI270335.1', 'KI270336.1', 'KI270337.1', 'KI270338.1', 'KI270340.1', 'KI270362.1', 'KI270363.1', 'KI270364.1', 'KI270366.1', 'KI270371.1', 'KI270372.1', 'KI270373.1', 'KI270374.1', 'KI270375.1', 'KI270376.1', 'KI270378.1', 'KI270379.1', 'KI270381.1', 'KI270382.1', 'KI270383.1', 'KI270384.1', 'KI270385.

In [11]:
dnabert_raw_data = {}
for key, value in loaded_dictionary.items():
    #display(value[['chr_name', 'start', 'end', 'strand', 'transcript_type', 'CHROM', 'START_POS', 'END_POS', 'ID', 'REF', 'ALT']])
    print("CODE STARTED")
    print(key, value.shape)
    #ins_del_df = value[(value['ALT'].str.len() > value['REF'].str.len()) | (value['REF'].str.len() > value['ALT'].str.len())]
    new_df, df_kmer = get_sequences(value, reference_fasta)
    dnabert_raw_data[key] = new_df
    
    #input()
    out_folder_path = output_path+"Patient_wise/"+ key
    print(out_folder_path, "is created")
    if not os.path.exists(out_folder_path):
        os.makedirs(out_folder_path)
    df_kmer.to_csv(out_folder_path + "/dev.tsv", sep="\t", index= False)

CODE STARTED
TCGA-02-2483-01A-01D-1494-08 (25840, 13)
(25840, 12)
&*
(51680, 2)
/data/projects/GDC_Cancer_Wise/New_data/Brain/Generated_files/DNABERT_Data/Germline/core_prom/Patient_wise/TCGA-02-2483-01A-01D-1494-08 is created
CODE STARTED
TCGA-CS-5396-01A-02D-1465-02 (17219, 13)
(17219, 12)
&*
(34438, 2)
/data/projects/GDC_Cancer_Wise/New_data/Brain/Generated_files/DNABERT_Data/Germline/core_prom/Patient_wise/TCGA-CS-5396-01A-02D-1465-02 is created
CODE STARTED
TCGA-EZ-7264-01A-11D-2024-08 (24860, 13)
(24860, 12)
&*
(49720, 2)
/data/projects/GDC_Cancer_Wise/New_data/Brain/Generated_files/DNABERT_Data/Germline/core_prom/Patient_wise/TCGA-EZ-7264-01A-11D-2024-08 is created
CODE STARTED
TCGA-16-1460-01A-01D-0932-09 (22090, 13)
(22090, 12)
&*
(44180, 2)
/data/projects/GDC_Cancer_Wise/New_data/Brain/Generated_files/DNABERT_Data/Germline/core_prom/Patient_wise/TCGA-16-1460-01A-01D-0932-09 is created
CODE STARTED
TCGA-CS-5395-01A-01D-1468-08 (32246, 13)
(32246, 12)
&*
(64492, 2)
/data/projec

In [12]:
with open(output_path+"/raw_{}_vcf_data.pkl".format(non_coding_region), "wb") as file:
    pickle.dump(dnabert_raw_data, file)

In [13]:
for key, value in dnabert_raw_data.items():
    print(dnabert_raw_data[key].shape[0] - dnabert_raw_data[key][['chr','Acceptor_start','Acceptor_end']].shape[0])
    if (dnabert_raw_data[key].shape[0]!=dnabert_raw_data[key][['chr','Acceptor_start','Acceptor_end']].shape[0]):
        print(key , " has instances where there are multiple variants")

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
